In [56]:
# (0, 0, 0, 0)
# (0, 1, 0, 1)
# (0, 2, 0, 2)
# (1, 2, 1, 0) # we turned down, so dc = 0 now
# (2, 2, 0, 1) # turned again
# (2, 3, 0, 2)
# (2, 3, 0, 3) # this is as far as we can go in this direction b/c dc = 3
# (1, 3, -1, 0)

"""
Just like yesterday, using complex numbers to store our position and direction.

To handle the movement constraints (min and max number of steps before a turn), most solutions check whether we are allowed to move one step in each of the directions (straight, left, and right).

Instead, we simply do each of "turn left and move min_steps", " turn left and move min_steps+1", ..., up to " turn right and move max_steps". As long as the destination is still on the map, each of these is valid. In code:

for dir in left, right:
    for steps in range(min_steps, max_steps+1):
        if pos + steps*dir in G:
            total_loss = sum(G[pos + step*dir] for step in range(1, step+1))
"""

In [57]:
import numpy as np


data = """2413432311323
3215453535623
3255245654254
3446585845452
4546657867536
1438598798454
4457876987766
3637877979653
4654967986887
4564679986453
1224686865563
2546548887735
4322674655533
"""

blocks = np.array([[int(n) for n in d] for d in data.strip().split("\n")])

blocks

array([[2, 4, 1, 3, 4, 3, 2, 3, 1, 1, 3, 2, 3],
       [3, 2, 1, 5, 4, 5, 3, 5, 3, 5, 6, 2, 3],
       [3, 2, 5, 5, 2, 4, 5, 6, 5, 4, 2, 5, 4],
       [3, 4, 4, 6, 5, 8, 5, 8, 4, 5, 4, 5, 2],
       [4, 5, 4, 6, 6, 5, 7, 8, 6, 7, 5, 3, 6],
       [1, 4, 3, 8, 5, 9, 8, 7, 9, 8, 4, 5, 4],
       [4, 4, 5, 7, 8, 7, 6, 9, 8, 7, 7, 6, 6],
       [3, 6, 3, 7, 8, 7, 7, 9, 7, 9, 6, 5, 3],
       [4, 6, 5, 4, 9, 6, 7, 9, 8, 6, 8, 8, 7],
       [4, 5, 6, 4, 6, 7, 9, 9, 8, 6, 4, 5, 3],
       [1, 2, 2, 4, 6, 8, 6, 8, 6, 5, 5, 6, 3],
       [2, 5, 4, 6, 5, 4, 8, 8, 8, 7, 7, 3, 5],
       [4, 3, 2, 2, 6, 7, 4, 6, 5, 5, 5, 3, 3]])

In [129]:
def is_forwards(dr, dc, mr, mc):
    return dr * mr + dc * mc > 0

def is_backwards(dr, dc, mr, mc):
    return dr * mr + dc * mc < 0

def is_orthogonal(dr, dc, mr, mc):
    return dr * mr + dc * mc == 0

def is_out_of_bounds(nr, nc, sr, sc):
    return not (0 <= nr < sr and 0 <= nc < sc)

def next_move(coord, move, shape):
    cr, cc, dr, dc = coord
    mr, mc = move
    nr, nc = (cr + mr, cc + mc)
    sr, sc = shape
    if is_out_of_bounds(nr, nc, sr, sc):
        return None
    if is_orthogonal(dr, dc, mr, mc):
        ndr, ndc = mr, mc
        return nr, nc, ndr, ndc
    elif is_backwards(dr, dc, mr, mc):
        return None
    elif is_forwards(dr, dc, mr, mc):
        ndr, ndc = mr + dr, mc + dc
        if abs(ndr) <= 3 and abs(ndc) <= 3:
            return nr, nc, ndr, ndc
        else:
            return None
    return None


stack = [(0, (0, 0, 0, 0))]
seen = set()
losses = {(0, 0, 0, 0): 0}
while stack:
    curr_loss, curr_coord = stack.pop()
    if curr_coord in seen:
        continue
    else:
        seen.add(curr_coord)
    
    if curr_coord[:2] == (12, 12):
        print(curr_loss, curr_coord)
    
    moves = [
        (-1, 0), (1, 0), (0, -1), (0, 1),
        (-2, 0), (2, 0), (0, -2), (0, 2),
        (-3, 0), (3, 0), (0, -3), (0, 3)
    ]
    next_coords = [
        next_move(curr_coord, m, blocks.shape) for m in moves
        if next_move(curr_coord, m, blocks.shape)
    ]
    for next_coord in next_coords:
        nr, nc, ndr, ndc = next_coord
        if next_coord in seen:
            continue
        
        # pdb.set_trace()
        route_loss = 0
        for i in range(1, ndr + 1):
            route_loss += blocks[cr + i, cc]
        for j in range(1, ndc + 1):
            route_loss += blocks[cr, cc + j]
        
        next_loss = curr_loss + route_loss
        stack.append((next_loss, next_coord))


68 (12, 12, 3, 0)
847 (12, 12, 0, 3)
2294 (12, 12, 0, 2)
2311 (12, 12, 2, 0)
2424 (12, 12, 0, 1)
2923 (12, 12, 1, 0)


In [125]:
blocks

array([[2, 4, 1, 3, 4, 3, 2, 3, 1, 1, 3, 2, 3],
       [3, 2, 1, 5, 4, 5, 3, 5, 3, 5, 6, 2, 3],
       [3, 2, 5, 5, 2, 4, 5, 6, 5, 4, 2, 5, 4],
       [3, 4, 4, 6, 5, 8, 5, 8, 4, 5, 4, 5, 2],
       [4, 5, 4, 6, 6, 5, 7, 8, 6, 7, 5, 3, 6],
       [1, 4, 3, 8, 5, 9, 8, 7, 9, 8, 4, 5, 4],
       [4, 4, 5, 7, 8, 7, 6, 9, 8, 7, 7, 6, 6],
       [3, 6, 3, 7, 8, 7, 7, 9, 7, 9, 6, 5, 3],
       [4, 6, 5, 4, 9, 6, 7, 9, 8, 6, 8, 8, 7],
       [4, 5, 6, 4, 6, 7, 9, 9, 8, 6, 4, 5, 3],
       [1, 2, 2, 4, 6, 8, 6, 8, 6, 5, 5, 6, 3],
       [2, 5, 4, 6, 5, 4, 8, 8, 8, 7, 7, 3, 5],
       [4, 3, 2, 2, 6, 7, 4, 6, 5, 5, 5, 3, 3]])